In [4]:
from urllib.request import urlopen   
from urllib.error import URLError, HTTPError
import pandas as pd
import os
import matplotlib.pyplot as plt
import sqlite3
import numpy as np
import seaborn as sns; sns.set(style="ticks", color_codes=True)

%matplotlib inline
pd.set_option('display.max_columns', None)

#Use the publicly available IMDB Datasets to build a model that predicts a movie’s average rating. Please
#create a github account and upload code for the analysis to the account. Make sure your code is well
#organized and documented with a readme or within a notebook so that we can follow your approach.
#Document your approach and present your results in the readme or notebook. Be prepared to discuss
#how you would improve your model (if you had additional time and resources) during your interview.

# Data Collection

In [5]:
# adapted from https://stackoverflow.com/questions/29537298/python-3-urllib-request-urlopen
def download_file(file_name, file_url):
    try:
        response = urlopen(file_url)
        try:
            output = open(file_name, 'wb')         
            output.write(response.read())
            output.close()
        except IOError as e:
            print ("I/O error({0}): {1}".format(e.errno, e.strerror))
        except ValueError:
            print ("ValueError")
        except:
            print ("Unexpected error:", sys.exc_info()[0])
            raise
    except HTTPError as e:
        print('Error code: ', e.code)
    except URLError as e:
        print('Reason: ', e.reason)
    else:
        print('Downloaded: ', file_url)

In [6]:
# documentation https://www.imdb.com/interfaces/
file_dict = {
    "name.basics.tsv.gz" : "https://datasets.imdbws.com/name.basics.tsv.gz",
    "title.akas.tsv.gz" : "https://datasets.imdbws.com/title.akas.tsv.gz",
    "title.basics.tsv.gz" : "https://datasets.imdbws.com/title.basics.tsv.gz",
    "title.crew.tsv.gz" : "https://datasets.imdbws.com/title.crew.tsv.gz",
    "title.episode.tsv.gz" : "https://datasets.imdbws.com/title.episode.tsv.gz",
    "title.principals.tsv.gz" : "https://datasets.imdbws.com/title.principals.tsv.gz",
    "title.ratings.tsv.gz" : "https://datasets.imdbws.com/title.ratings.tsv.gz"
}

#for file_name, file_url in file_dict.items():
    #print(file_name, ":", file_url)
    #download_file(file_name,file_url)

In [7]:
name_basics_df = pd.read_csv('name.basics.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
name_basics_df.head(3)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0043044,tt0050419"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0071877,tt0038355,tt0037382"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,producer","tt0057345,tt0059956,tt0049189,tt0054452"


In [8]:
title_akas_df = pd.read_csv('title.akas.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
title_akas_df.head(3)

F:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
1,tt0000001,2,Καρμενσίτα,GR,\N,\N,\N,0
2,tt0000001,3,Карменсита,RU,\N,\N,\N,0


In [9]:
#heavily skewed torwards null
title_akas_df.groupby('types')['types'].count()

types
\N                  2655233
alternative           58714
dvd                   19998
dvdimdbDisplay           3
dvdvideo                 2
dvdworking               1
festival              13379
festivalworking          2
imdbDisplay          477491
original             391832
tv                    16090
tvvideo                  6
tvworking                4
video                 15538
videoworking             5
working               42426
Name: types, dtype: int64

In [10]:
title_basics_df = pd.read_csv('title.basics.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
title_basics_df.head(3)

F:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"


In [11]:
# not needed - taken care of in the principals
#title_crew_df = pd.read_csv('title.crew.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
#title_crew_df.tail(3)

In [12]:
# not needed due to movie only focus
#title_episode_df = pd.read_csv('title.episode.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
#title_episode_df.head()

In [13]:
title_principals_df = pd.read_csv('title.principals.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
title_principals_df.head(3)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Herself""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N


In [14]:
title_ratings_df = pd.read_csv('title.ratings.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
title_ratings_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.8,1523
1,tt0000002,6.3,185
2,tt0000003,6.6,1165
3,tt0000004,6.3,113
4,tt0000005,6.2,1879


# Data Processing

In [87]:
# Business problem is to predict movie's average rating. Since we are doing only movies,
# we can select only movies using titleType = "movie"
title_movies_df = title_basics_df[title_basics_df['titleType'] == 'movie']
# drop unneeded columns
title_movies_df = title_movies_df.drop(['titleType','endYear','primaryTitle'], axis = 1)
title_movies_df.head(3)

,tconst,originalTitle,isAdult,startYear,runtimeMinutes,genres
8,tt0000009,Miss Jerry,0,1894,45,Romance
145,tt0000147,The Corbett-Fitzsimmons Fight,0,1897,20,"Documentary,News,Sport"
332,tt0000335,Soldiers of the Cross,0,1900,\N,"Biography,Drama"


In [88]:
# add in dependent var (averageRating)
title_movies_df = title_movies_df.join(title_ratings_df.set_index('tconst'), on='tconst')
title_movies_df = title_movies_df.dropna(subset = ['averageRating','numVotes']) # remove movies where there is no rating or info about numvotes

In [89]:
title_akas_deduped_df = title_akas_df[['titleId','region']].drop_duplicates()

In [90]:
# top 25 out of 250+ regions can capture most of regions
region_keys = title_akas_deduped_df.groupby('region')['titleId'].count() \
    .reset_index(name='count').sort_values(['count'], ascending=False) 
region_keys['pct_total'] = region_keys['count'] / region_keys['count'].sum()
region_keys_filter = region_keys.head(25)['region']
region_keys.head(10)

,region,count,pct_total
213,US,879874,0.257069
244,\N,391761,0.114459
72,GB,250854,0.073291
70,FR,138906,0.040584
65,ES,106488,0.031112
105,JP,103830,0.030336
54,DE,100863,0.029469
236,XWW,99846,0.029172
35,CA,86822,0.025366
10,AR,82667,0.024152


In [91]:
# create dataframe with movies and the regions they were released in
title_akas_transpose = title_akas_deduped_df[title_akas_deduped_df['region'].isin(region_keys_filter)]
title_akas_transpose = title_akas_transpose.groupby(['titleId','region'], as_index=False).size().unstack('region', fill_value = 0)
title_akas_transpose = title_akas_transpose.reset_index().set_index('titleId').rename_axis(None, axis=1)
title_akas_transpose.head(2)

,AR,AU,BR,CA,DE,DK,ES,FI,FR,GB,GR,HU,IN,IT,JP,MX,PL,PT,RU,SE,TR,US,XWG,XWW,\N
titleId,,,,,,,,,,,,,,,,,,,,,,,,,
tt0000001,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,1
tt0000002,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1


In [92]:
# join region into main df
title_movies_df = title_movies_df.join(title_akas_transpose, on='tconst').fillna(0)

In [94]:
#replace \N with null so it is properly ignored
title_movies_df['genres'] = title_movies_df['genres'].replace("\\N", np.nan)

In [95]:
# need to split genres into separate columns, ex: Documentary,News,Sport 
genre_df = title_movies_df.set_index('tconst')['genres'].str.get_dummies(',')
# join genre to main df
title_movies_df = title_movies_df.join(genre_df, on='tconst').fillna(0)
title_movies_df = title_movies_df.drop(['genres'], axis = 1)

In [96]:
# issue - joining converts integers into floats, we must convert them back
# https://github.com/pandas-dev/pandas/issues/9958
title_movies_df[list(title_akas_transpose.columns) ] = title_movies_df[list(title_akas_transpose.columns) ].astype(int)
title_movies_df[list(genre_df.columns) ] = title_movies_df[list(genre_df.columns) ].astype(int)

In [97]:
# remove movies with null / invalid start year and low sample size
title_movies_df = title_movies_df[(title_movies_df['startYear'] != '\\N') & (title_movies_df['numVotes'] > 30)]
#convert year back to int so we can do comparisons
title_movies_df['startYear'] = title_movies_df['startYear'].astype(int)

In [98]:
title_movies_df.sort_values(['numVotes']).tail(3)

,tconst,originalTitle,isAdult,startYear,runtimeMinutes,averageRating,numVotes,AR,AU,BR,CA,DE,DK,ES,FI,FR,GB,GR,HU,IN,IT,JP,MX,PL,PT,RU,SE,TR,US,XWG,XWW,\N,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
1639913,tt1375666,Inception,0,2010,148,8.8,1852716.0,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
450824,tt0468569,The Dark Knight,0,2008,152,9.0,2079209.0,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
108772,tt0111161,The Shawshank Redemption,0,1994,142,9.3,2113420.0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [99]:
# join list of principals with list of movies to reduce number of rows needed to process
desired_principals = title_principals_df.set_index('tconst') \
    .join(title_movies_df[['tconst']].set_index('tconst'),how='inner').reset_index()

In [100]:
# get top 2 directors, writers, actors and get the number of ranked films + weighted avg rating
desired_roles = pd.DataFrame(['actor','writer','director'], columns = ['category'])
desired_principals = desired_principals[(desired_principals['category'].isin(desired_roles['category']))] \
                                        .sort_values(['tconst','ordering'], ascending=[True,True])

In [101]:
desired_principals['rank'] = desired_principals.groupby(['tconst','category']).cumcount()+1 #rank by role type
desired_principals = desired_principals[desired_principals['rank'] <= 2] \
    .sort_values(['tconst','category','rank'], ascending=[True,True,True]) # only keep top two of each
desired_principals = desired_principals.drop(['ordering','job','characters'], axis = 1)
desired_principals['colname'] = desired_principals['category'] +"_" + desired_principals['rank'].astype(str)

In [102]:
desired_principals.head(3)

,tconst,nconst,category,rank,colname
1,tt0000009,nm0183823,actor,1,actor_1
3,tt0000009,nm0085156,director,1,director_1
8,tt0000147,nm0714557,director,1,director_1


In [103]:
# create df with movie and principal 1st and 2nd actor, director, and writer
principal_df = desired_principals[['tconst','nconst','colname']].pivot(index='tconst', columns = 'colname')
principal_df.columns = principal_df.columns.droplevel()
principal_df.head(3)

colname,actor_1,actor_2,director_1,director_2,writer_1,writer_2
tconst,,,,,,
tt0000009,nm0183823,None,nm0085156,None,None,None
tt0000147,None,None,nm0714557,None,None,None
tt0000335,nm1012612,nm1011210,nm0095714,nm0675140,None,None


In [104]:
#join principal people to main dataframe
title_movies_df = title_movies_df.set_index('tconst') \
    .join(principal_df,how='left').reset_index()
title_movies_df.tail(2)

,tconst,originalTitle,isAdult,startYear,runtimeMinutes,averageRating,numVotes,AR,AU,BR,CA,DE,DK,ES,FI,FR,GB,GR,HU,IN,IT,JP,MX,PL,PT,RU,SE,TR,US,XWG,XWW,\N,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,actor_1,actor_2,director_1,director_2,writer_1,writer_2
136418,tt9911774,Padmavyuhathile Abhimanyu,0,2019,130,8.3,368.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nm2649680,nm1428724,nm10536451,None,nm10536453,nm10536454
136419,tt9914286,Sokagin Çocuklari,0,2019,98,8.7,141.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nm4394529,nm2456249,None,None,nm1902682,None


In [105]:
# just discovered I can query dataframes
conn = sqlite3.connect(":memory:")
title_movies_df.to_sql("title_movies_sql", conn, index=False)

In [106]:
# getting weighted average of ratings for principals movies in same role 1 to 3 years prior
person_ratings_qry = """
with person as (
    select tconst, actor_1 as nconst, startYear,Numvotes, averageRating, 'actor' as roletype, 'actor_1_attr' as colname
    from title_movies_sql
    where nconst is not NULL
    union
    select tconst, actor_2 as nconst, startYear,Numvotes, averageRating, 'actor' as roletype, 'actor_2_attr' as colname
    from title_movies_sql
    where nconst is not NULL
    union
    select tconst, director_1 as nconst, startYear,Numvotes, averageRating, 'director' as roletype, 'director_1_attr' as colname
    from title_movies_sql
    where nconst is not NULL
    union
    select tconst, director_2 as nconst, startYear,Numvotes, averageRating, 'director' as roletype, 'director_2_attr' as colname
    from title_movies_sql
    where nconst is not NULL
    union
    select tconst, writer_1 as nconst, startYear,Numvotes, averageRating, 'writer' as roletype, 'writer_1_attr' as colname
    from title_movies_sql
    where nconst is not NULL
    union
    select tconst, writer_2 as nconst, startYear,Numvotes, averageRating, 'writer' as roletype, 'writer_2_attr' as colname
    from title_movies_sql
    where nconst is not NULL
)
SELECT person.tconst, person.nconst, person.roletype, person.colname,
sum(prev.Numvotes) as Numvotes,
sum(prev.Numvotes * prev.averageRating) / sum(prev.numVotes) as weightedAvg
FROM person
LEFT JOIN person prev
    on prev.nconst = person.nconst
    and person.startYear - prev.startYear between 1 and 3
    and person.roletype = prev.roletype
group by person.tconst, person.nconst, person.roletype, person.colname
"""
person_ratings_df = pd.read_sql_query(person_ratings_qry,conn)
person_ratings_df.tail()

,tconst,nconst,roletype,colname,Numvotes,weightedAvg
511178,tt9911774,nm1428724,actor,actor_2_attr,203.0,6.004433
511179,tt9911774,nm2649680,actor,actor_1_attr,NaN,NaN
511180,tt9914286,nm1902682,writer,writer_1_attr,NaN,NaN
511181,tt9914286,nm2456249,actor,actor_2_attr,NaN,NaN
511182,tt9914286,nm4394529,actor,actor_1_attr,NaN,NaN


In [107]:
# pivot dataframe to join to main df
person_ratings_pivot = person_ratings_df[['tconst','nconst','colname','Numvotes','weightedAvg']].pivot(index='tconst', columns = 'colname')
person_ratings_pivot = person_ratings_pivot[['weightedAvg','Numvotes']]
person_ratings_pivot = person_ratings_pivot.round(2) 
person_ratings_pivot.tail(3)

weightedAvg                                               \
colname   actor_1_attr actor_2_attr director_1_attr director_2_attr   
tconst                                                                
tt9905462         5.88         6.56             NaN             NaN   
tt9911774          NaN         6.00             NaN             NaN   
tt9914286          NaN          NaN             NaN             NaN   

                                          Numvotes               \
colname   writer_1_attr writer_2_attr actor_1_attr actor_2_attr   
tconst                                                            
tt9905462           NaN           NaN       2531.0       1217.0   
tt9911774           NaN           NaN          NaN        203.0   
tt9914286           NaN           NaN          NaN          NaN   

                                                                       
colname   director_1_attr director_2_attr writer_1_attr writer_2_attr  
tconst                                                                 
tt9905462             NaN             NaN           NaN           NaN  
tt9911774             NaN             NaN           NaN           NaN  
tt9914286             NaN             NaN           NaN           NaN

In [108]:
# merge column names 
person_ratings_pivot.columns = [['_'.join(col).rstrip('_') for col in person_ratings_pivot.columns.values]]
person_ratings_pivot.tail()

,weightedAvg_actor_1_attr,weightedAvg_actor_2_attr,weightedAvg_director_1_attr,weightedAvg_director_2_attr,weightedAvg_writer_1_attr,weightedAvg_writer_2_attr,Numvotes_actor_1_attr,Numvotes_actor_2_attr,Numvotes_director_1_attr,Numvotes_director_2_attr,Numvotes_writer_1_attr,Numvotes_writer_2_attr
tconst,,,,,,,,,,,,
tt9904844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt9905412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt9905462,5.88,6.56,NaN,NaN,NaN,NaN,2531.0,1217.0,NaN,NaN,NaN,NaN
tt9911774,NaN,6.00,NaN,NaN,NaN,NaN,NaN,203.0,NaN,NaN,NaN,NaN
tt9914286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
#join principal people ratings to main dataframe
title_movies_df = title_movies_df.set_index('tconst') \
    .join(person_ratings_pivot,how='left').reset_index()

## Impute!

In [111]:
# impute it with the mean for that year
title_movies_df['weightedAvg_director_1_attr'] = title_movies_df.groupby(['startYear'])['weightedAvg_director_1_attr']\
    .transform(lambda x: x.fillna(x.mean())) 
title_movies_df['weightedAvg_director_2_attr'] = title_movies_df.groupby(['startYear'])['weightedAvg_director_2_attr']\
    .transform(lambda x: x.fillna(x.mean()))     
# for movies without second principal, set avg using first principal, since first principal will have more influence
title_movies_df['weightedAvg_director_2_attr'] = title_movies_df['weightedAvg_director_2_attr'].fillna(title_movies_df['weightedAvg_director_1_attr'])

#numvotes
title_movies_df['Numvotes_director_1_attr'] = title_movies_df.groupby(['startYear'])['Numvotes_director_1_attr']\
    .transform(lambda x: x.fillna(x.mean())) 
title_movies_df['Numvotes_director_2_attr'] = title_movies_df.groupby(['startYear'])['Numvotes_director_2_attr']\
    .transform(lambda x: x.fillna(x.mean()))     
title_movies_df['Numvotes_director_2_attr'] = title_movies_df['Numvotes_director_2_attr'].fillna(title_movies_df['Numvotes_director_1_attr'])

In [112]:
# impute it with the mean for that year
title_movies_df['weightedAvg_actor_1_attr'] = title_movies_df.groupby(['startYear'])['weightedAvg_actor_1_attr']\
    .transform(lambda x: x.fillna(x.mean())) 
title_movies_df['weightedAvg_actor_2_attr'] = title_movies_df.groupby(['startYear'])['weightedAvg_actor_2_attr']\
    .transform(lambda x: x.fillna(x.mean()))     
# for movies without second principal, set avg using first principal, since first principal will have more influence
title_movies_df['weightedAvg_actor_2_attr'] = title_movies_df['weightedAvg_actor_2_attr'].fillna(title_movies_df['weightedAvg_actor_1_attr'])

#numvotes
title_movies_df['Numvotes_actor_1_attr'] = title_movies_df.groupby(['startYear'])['Numvotes_actor_1_attr']\
    .transform(lambda x: x.fillna(x.mean())) 
title_movies_df['Numvotes_actor_2_attr'] = title_movies_df.groupby(['startYear'])['Numvotes_actor_2_attr']\
    .transform(lambda x: x.fillna(x.mean()))     
title_movies_df['Numvotes_actor_2_attr'] = title_movies_df['Numvotes_actor_2_attr'].fillna(title_movies_df['Numvotes_actor_1_attr'])

In [113]:
# impute it with the mean for that year
title_movies_df['weightedAvg_writer_1_attr'] = title_movies_df.groupby(['startYear'])['weightedAvg_writer_1_attr']\
    .transform(lambda x: x.fillna(x.mean())) 
title_movies_df['weightedAvg_writer_2_attr'] = title_movies_df.groupby(['startYear'])['weightedAvg_writer_2_attr']\
    .transform(lambda x: x.fillna(x.mean()))     
# for movies without second principal, set avg using first principal, since first principal will have more influence
title_movies_df['weightedAvg_writer_2_attr'] = title_movies_df['weightedAvg_writer_2_attr'].fillna(title_movies_df['weightedAvg_writer_1_attr'])

#numvotes
title_movies_df['Numvotes_writer_1_attr'] = title_movies_df.groupby(['startYear'])['Numvotes_writer_1_attr']\
    .transform(lambda x: x.fillna(x.mean())) 
title_movies_df['Numvotes_writer_2_attr'] = title_movies_df.groupby(['startYear'])['Numvotes_writer_2_attr']\
    .transform(lambda x: x.fillna(x.mean()))     
title_movies_df['Numvotes_writer_2_attr'] = title_movies_df['Numvotes_writer_2_attr'].fillna(title_movies_df['Numvotes_writer_1_attr'])

In [114]:
title_movies_df.head()

,tconst,originalTitle,isAdult,startYear,runtimeMinutes,averageRating,numVotes,AR,AU,BR,CA,DE,DK,ES,FI,FR,GB,GR,HU,IN,IT,JP,MX,PL,PT,RU,SE,TR,US,XWG,XWW,\N,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,actor_1,actor_2,director_1,director_2,writer_1,writer_2,weightedAvg_actor_1_attr,weightedAvg_actor_2_attr,weightedAvg_director_1_attr,weightedAvg_director_2_attr,weightedAvg_writer_1_attr,weightedAvg_writer_2_attr,Numvotes_actor_1_attr,Numvotes_actor_2_attr,Numvotes_director_1_attr,Numvotes_director_2_attr,Numvotes_writer_1_attr,Numvotes_writer_2_attr
0,tt0000009,Miss Jerry,0,1894,45,5.5,80.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,nm0183823,None,nm0085156,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0000147,The Corbett-Fitzsimmons Fight,0,1897,20,5.2,301.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,None,None,nm0714557,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0000335,Soldiers of the Cross,0,1900,\N,6.3,39.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nm1012612,nm1011210,nm0095714,nm0675140,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0000574,The Story of the Kelly Gang,0,1906,70,6.2,515.0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nm0846894,nm3002376,nm0846879,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0000679,The Fairylogue and Radio-Plays,0,1908,120,4.8,33.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nm0000875,nm0122665,nm0091767,nm0877783,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
# replace existing \N with NAN so we can easily impute with mean using fillna 
title_movies_df['runtimeMinutes'] = title_movies_df['runtimeMinutes'].replace("\\N", np.nan)

title_movies_df['runtimeMinutes'] = title_movies_df['runtimeMinutes'].astype(float)
title_movies_df['runtimeMinutes'] = title_movies_df.groupby(['startYear'])['runtimeMinutes'].apply(lambda x: x.fillna(x.mean()))

In [116]:
#drop films before 1913 - most movies prior were very basic and limited. https://en.wikipedia.org/wiki/Classical_Hollywood_cinema#Maturation_of_the_silents_(1913%E2%80%93late_1920s)
title_movies_df = title_movies_df[ (title_movies_df['startYear'] >= 1913)]

# drop null genre column
title_movies_df = title_movies_df.drop(['\\N'], axis = 1)

title_movies_df.head(5)

,tconst,originalTitle,isAdult,startYear,runtimeMinutes,averageRating,numVotes,AR,AU,BR,CA,DE,DK,ES,FI,FR,GB,GR,HU,IN,IT,JP,MX,PL,PT,RU,SE,TR,US,XWG,XWW,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,actor_1,actor_2,director_1,director_2,writer_1,writer_2,weightedAvg_actor_1_attr,weightedAvg_actor_2_attr,weightedAvg_director_1_attr,weightedAvg_director_2_attr,weightedAvg_writer_1_attr,weightedAvg_writer_2_attr,Numvotes_actor_1_attr,Numvotes_actor_2_attr,Numvotes_director_1_attr,Numvotes_director_2_attr,Numvotes_writer_1_attr,Numvotes_writer_2_attr
15,tt0002423,Madame DuBarry,0,1919,85.0,6.9,693.0,0,0,1,0,1,1,0,1,1,0,0,1,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nm0417837,nm0509573,nm0523932,None,nm0266183,nm0473134,5.84,5.91,6.43,6.48,6.2,6.42,1011.0,1253.0,2381.0,1963.0,242.0,2048.0
16,tt0002445,Quo Vadis?,0,1913,120.0,6.2,235.0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nm0637019,nm0784988,nm0345500,None,nm0797197,None,7.20,7.20,5.80,5.80,5.6,5.60,266.0,266.0,183.0,183.0,206.0,206.0
19,tt0002628,Der Andere,0,1913,48.0,5.7,62.0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,nm0060168,nm0361518,nm0533048,None,nm0511317,nm1857018,7.20,7.20,5.80,5.80,5.6,5.60,266.0,266.0,183.0,183.0,206.0,206.0
20,tt0002646,Atlantis,0,1913,121.0,6.8,304.0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nm0299761,nm0491498,nm0088881,None,nm1834296,nm0369458,7.20,7.20,5.48,5.48,5.6,5.60,266.0,266.0,156.0,156.0,206.0,206.0
21,tt0002767,The Count of Monte Cristo,0,1913,69.0,5.4,35.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nm0642189,nm0534259,nm0325448,nm0692105,nm0241416,nm0270007,7.20,7.20,5.80,5.80,5.6,5.60,266.0,266.0,183.0,183.0,206.0,206.0


In [117]:
# save data to tsv 
title_movies_df.to_csv('imdb.agg.tsv', sep="\t",index=False, encoding='utf-8')